In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

cifar_10_path = kagglehub.competition_download('cifar-10')

print('Data source import complete.')


Data source import complete.


In [ ]:
cifar_10_path

'/root/.cache/kagglehub/competitions/cifar-10'

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from time import time as timer
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import random_split, Dataset, DataLoader

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
labels_csv = pd.read_csv("/root/.cache/kagglehub/competitions/cifar-10/trainLabels.csv")
labels_csv

,id,label
0,1,frog
1,2,truck
2,3,truck
3,4,deer
4,5,automobile
...,...,...
49995,49996,bird
49996,49997,frog
49997,49998,truck
49998,49999,automobile


In [ ]:
label_mapping = {label: idx for idx, label in enumerate(labels_csv['label'].unique())}

labels_csv.rename({"label": "label_txt"}, axis=1, inplace=True)
labels_csv['label'] = labels_csv['label_txt'].map(label_mapping)

print(label_mapping, labels_csv)

{'frog': 0, 'truck': 1, 'deer': 2, 'automobile': 3, 'bird': 4, 'horse': 5, 'ship': 6, 'cat': 7, 'dog': 8, 'airplane': 9}           id   label_txt  label
0          1        frog      0
1          2       truck      1
2          3       truck      1
3          4        deer      2
4          5  automobile      3
...      ...         ...    ...
49995  49996        bird      4
49996  49997        frog      0
49997  49998       truck      1
49998  49999  automobile      3
49999  50000  automobile      3

[50000 rows x 3 columns]


In [ ]:
!pip install py7zr

import py7zr

archive_path = '/root/.cache/kagglehub/competitions/cifar-10/train.7z'

extract_to = ''

with py7zr.SevenZipFile(archive_path, mode='r') as z:
    z.extractall(path=extract_to)

print(f"Архив распакован в {extract_to}")

Архив распакован в 


In [ ]:
train_dir = "train"
# test_dir = "/kaggle/working/test"

train_imgs = os.listdir(train_dir)

img1 = Image.open(train_dir + "/" + train_imgs[0])
img1.size

(32, 32)

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
])

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, images_dir, labels_csv, transform=None):
        self.transforms = transform
        self.imgs_dir = images_dir
        self.labels_csv = labels_csv

    def __len__(self):
        return len(self.labels_csv)

    def __getitem__(self, idx):
        img_name = str(self.labels_csv.iloc[idx, 0]) + ".png"
        label = self.labels_csv.loc[idx, "label"]

        img_path = os.path.join(self.imgs_dir, img_name)

        img = Image.open(img_path).convert("RGB")

        if self.transforms:
            img = self.transforms(img)

        return img, label

In [ ]:
train_dataset = CustomDataset(
    images_dir = "train", labels_csv = labels_csv,
    transform = train_transform
)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
from tqdm import tqdm

import torch.nn as nn
import torch.nn.functional as F

class CNNModel(nn.Module):
    def __init__(self, num_classes=10):
        super(CNNModel, self).__init__()

        self.conv_block = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # [B, 32, 32, 32]
            nn.ReLU(),
            nn.MaxPool2d(2),  # [B, 32, 16, 16]

            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # [B, 64, 16, 16]
            nn.ReLU(),
            nn.MaxPool2d(2),  # [B, 64, 8, 8]

            nn.Conv2d(64, 128, kernel_size=3, padding=1),  # [B, 128, 8, 8]
            nn.ReLU(),
            nn.MaxPool2d(2),  # [B, 128, 4, 4]
        )

        self.fc_block = nn.Sequential(
            nn.Flatten(),  # [B, 128*4*4]
            nn.Linear(128 * 4 * 4, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.conv_block(x)
        x = self.fc_block(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNModel(num_classes=10).to(device)

cr = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

corrects = 0
epochs = 10

for i in range(epochs):
    cost = 0
    for image, answer in tqdm(train_loader):
        image = image.to(device)
        answer = answer.to(device)
        # print(image, answer)

        optimizer.zero_grad()
        predictions = model(image)
        error = cr(predictions, answer)
        error.backward()
        optimizer.step()
        cost += error.item()

        if i == epochs - 1:
            corrects += (predictions.argmax(1) == answer).sum().item()

    print(f"Epoch {i+1}, Loss: {cost / len(train_loader):.4f}")

total_samples = len(train_loader.dataset)
print("Accuracy:", corrects, "/", total_samples, "=", corrects / total_samples)

100%|██████████| 1563/1563 [00:35<00:00, 44.12it/s]


Epoch 1, Loss: 1.4771


100%|██████████| 1563/1563 [00:33<00:00, 46.60it/s]


Epoch 2, Loss: 1.0944


100%|██████████| 1563/1563 [00:34<00:00, 45.86it/s]


...

Epoch 8, Loss: 0.6164


100%|██████████| 1563/1563 [00:34<00:00, 45.65it/s]


Epoch 9, Loss: 0.5812


100%|██████████| 1563/1563 [00:34<00:00, 45.20it/s]

Epoch 10, Loss: 0.5527
Accuracy: 40451 / 50000 = 0.80902


In [ ]:
train_split = int(0.8 * len(train_dataset))
val_split = int(0.2 * len(train_dataset))
train_split, val_split

train_dataset, val_dataset = random_split(train_dataset, [train_split, val_split])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
!pip install py7zr

import py7zr

archive_path = '/root/.cache/kagglehub/competitions/cifar-10/test.7z'

extract_to = ''

with py7zr.SevenZipFile(archive_path, mode='r') as z:
    z.extractall(path=extract_to)

print(f"Архив распакован в {extract_to}")

Архив распакован в 


In [ ]:
class CustomTestDataset(Dataset):
    def __init__(self, images_dir, transform=None):
        self.images_dir = images_dir
        self.transform = transform
        self.img_files = sorted(os.listdir(images_dir))

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.images_dir, self.img_files[idx])
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, self.img_files[idx]


test_dataset = CustomTestDataset("test", test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
model.eval()

CNNModel(
  (conv_block): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_block): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=2048, out_features=256, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=256, out_features=10, bias=True)
  )
)

In [ ]:
all_preds = []
img_idxs = []

with torch.no_grad():
    for images, idxs in tqdm(test_loader):
        images = images.to(device)

        outputs = model(images)

        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        img_idxs.extend(idxs)

100%|██████████| 9375/9375 [02:25<00:00, 64.29it/s]


In [ ]:
idxs_to_classes = {v: k for k, v in label_mapping.items()}

predicted_labels = [idxs_to_classes[pred] for pred in all_preds]

submission_df = pd.DataFrame({
    "id": [int(file.split(".")[0]) for file in img_idxs],
    "label": predicted_labels
})

submission_df = submission_df.sort_values(by="id")

submission_df.to_csv("submission.csv", index=False)

In [ ]:
submission_df

,id,label
0,1,deer
111111,2,airplane
222222,3,automobile
233334,4,ship
244445,5,airplane
...,...,...
222218,299996,airplane
222219,299997,deer
222220,299998,deer
222221,299999,frog
